In [115]:
using Plots
using PyCall
using ControlSystems

gr()

Plots.GRBackend()

In [116]:
zc=0.35
g=9.8
dt=0.001
L=0.14
#歩幅
x=0.1


#支持時間
Tsup=Int32(0.8/dt)
#全脚期間
Tdl=Int32(0.2/dt)

#全体のステップ数
k_max=Int32(1+(Tsup*6))
#目標予見ステップ
M=800

#支持脚位置
#sx=[0.0 0.0 0.3 0.6 0.9 0.9]
#sy=[0.0 0.2 0.0 0.2 0.0 0.2]

#real support point
real_sx=[0.0; (1/2)*sqrt(3)*L; (1/2)*sqrt(3)*L+x; (1/2)*sqrt(3)*L+x; sqrt(3)*L+x; sqrt(3)*L+2x; sqrt(3)*L+2x; (3/2)*sqrt(3)*L+2x]
real_sy=[L/2; 0.0; L; L/2; 0.0; L; L/2; 0.0]

#Virtual support point
sx=[(1/2)*(sqrt(3)*L+x) 
    (1/2)*sqrt(3)*L+x 
    (3/4)*sqrt(3)*L+x 
    sqrt(3)*L+(3/2)*x 
    sqrt(3)*L+2*x 
    (5/4)*sqrt(3)*L+2*x]
sy=[L/2 
    3*L/4 
    L/4 
    L/2 
    3*L/4 
    L/4]

#目標ZMPベクトル
px_ref=[[sx[1] for s in 1:Tsup]
        [sx[2] for s in 1:Tsup]
        [sx[3] for s in 1:Tsup]
        [sx[4] for s in 1:Tsup]
        [sx[5] for s in 1:Tsup]
        [sx[6] for s in 1:Tsup+1+M]]

py_ref=[[sy[1] for s in 1:Tsup]
        [sy[2] for s in 1:Tsup]
        [sy[3] for s in 1:Tsup]
        [sy[4] for s in 1:Tsup]
        [sy[5] for s in 1:Tsup]
        [sy[6] for s in 1:Tsup+1+M]]

#計算上のZMPベクトル
px=zeros(Float64,k_max) 
py=zeros(Float64,k_max) 
k_max

4801

In [117]:
A=[1 dt dt^2/2
    0 1 dt
    0 0 1]
B=[dt^3/6
    dt^2/2
    dt]
C=[1 0 -zc/g]

Q=[100 0 0 0
    0 10 0 0
    0 0 1 0
    0 0 0 1]
R=1.0

#エラーシステム

#内部状態ベクトル
x=[ sx[1]+0.001
    0.0
    0.0]

y=[ L/2+0.01
    0.0
    0.0]

φ=[1 -C*A
    zeros(Float64,3) A]
G=[-C*B
    B]
Gr=[1.0
    0.0
    0.0
    0.0]

4-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0

In [118]:
k=0

t=[i for i in 1:k_max]
x_plot_log=zeros(Float64,k_max)
x_plot_log[k+1]=x[1] #plotするために記録
xv_plot_log=zeros(Float64,k_max)
xv_plot_log[k+1]=x[2]
xa_plot_log=zeros(Float64,k_max)
xa_plot_log[k+1]=x[3]

t=[i for i in 1:k_max]
y_plot_log=zeros(Float64,k_max)
y_plot_log[k+1]=y[1] #plotするために記録
yv_plot_log=zeros(Float64,k_max)
yv_plot_log[k+1]=y[2]
ya_plot_log=zeros(Float64,k_max)
ya_plot_log[k+1]=y[3] 


xdu=0
xdR=px_ref[k+1]-px[1] #dR[K+1]
ydu=0
ydR=py_ref[k+1]-py[1] #dR[K+1]

xe=px[1]-C*x
ye=py[1]-C*y


dx=[0; 0; 0]
dy=[0; 0; 0]

X=φ*[xe; dx]+G*xdu+Gr*xdR
Y=φ*[ye; dy]+G*ydu+Gr*ydR


old_x=x
old_y=y

xu=0
yu=0

P=dare(φ,G,Q,R)

ξ=(eye(4)-G*(R+G'*P*G)[1]^(-1)*G'*P)*φ

xsum=0
ysum=0
for j in 1:M
    Fr=-(R+G'*P*G)[1]^(-1)*G'*(ξ')^(j-1)*P*Gr
    xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1+1]) #K=0のみ+1
    ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1+1]) #K=0のみ+1
end
F=-(R+G'*P*G)[1]^(-1)*G'*P*φ
xdu=F*X+xsum
ydu=F*Y+ysum

xu+=xdu
yu+=ydu

1-element Array{Float64,1}:
 -0.0981879

In [119]:
for i in 1:k_max-1  
    #求めた入力を用いてk+1の状態を計算する
    px[k+1]=(C*x)[1]
    py[k+1]=(C*y)[1]
    
    x=A*x+B*xu[1]
    y=A*y+B*yu[1]
    
    k+=1
    
    x_plot_log[k+1]=x[1] #plotするために記録
    xv_plot_log[k+1]=x[2]
    xa_plot_log[k+1]=x[3]
    
    y_plot_log[k+1]=y[1] #plotするために記録
    yv_plot_log[k+1]=y[2]
    ya_plot_log[k+1]=y[3]
    
    dx=x-old_x
    dy=y-old_y
    
    xdR=px_ref[k+1]-px_ref[k]
    ydR=py_ref[k+1]-py_ref[k]
    
    
    xe=px_ref[k]-C*x
    ye=py_ref[k]-C*y
    
    X=φ*[xe; dx]+G*xdu[1]+Gr*xdR
    Y=φ*[ye; dy]+G*ydu[1]+Gr*ydR
    
    old_x=x
    old_y=y
    
    xsum=0
    ysum=0
    for j in 1:M
        Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
        xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1])
        ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1])
    end
    F=-(R+G'*P*G)*G'*P*φ
    xdu=F*X+xsum
    ydu=F*Y+ysum
    
    xu+=xdu
    yu+=ydu
end
    

In [120]:
plot(t,px_ref)
plot!(t,px)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
<polyline clip-path="url(#clip1202)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,246.322 39.4852,246.322 39.6004,246.322 39.7156,246.322 39.8307,246.322 39.9459,246.322 40.061,246.322 40.1762,246.322 40.2913,246.322 40.4065,246.322 
 40.5217,246.322 40.6368,246.322 40.752,246.322 40.8671,246.322 40.9823,246.322 41.0974,246.322 41.2126,246.322 41.3278,246.322 41.4429,246.322 41.5581,246.322 
 41.6732,246.322 41.7884,246.322 41.9035,246.322 42.0187,246.322 42.1339,246.322 42.249,246.322 42.3642,246.322 42.4793,246.322 42.5945,246.322 42.7096,246.322 
 42.8248,246.322 42.94,246.322 43.0551,246.322 43.1703,246.322 43.2854,246.322 43.4006,246.322 43.5157,246.322 43.6309,246.322 43.7461,246.322 43.8612,246.322 
 43.9764,246.322 44.0915,246.322 44.2067,246.322 44.3219,246.322 44.437,246.322 44.5522,246.322 44.6673,246.322 44.7825,246.322 44.8976,246.322 45.0128,246.322 
 45.128,246.322 45.2431,246.322 45.3583,246.322 45.4734,246.322 45.5886,246.322 45.7037,246.322 45.8189,246.322 45.9341,246.322 46.0492,246.322 46.1644,246.322 
 46.2795,246.322 46.3947,246.322 46.5098,246.322 46.625,246.322 46.7402,246.322 46.8553,246.322 46.9705,246.322 47.0856,246.322 47.2008,246.322 47.3159,246.322 
 47.4311,246.322 47.5463,246.322 47.6614,246.322 47.7766,246.322 47.8917,246.322 48.0069,246.322 48.122,246.322 48.2372,246.322 48.3524,246.322 48.4675,246.322 
 48.5827,246.322 48.6978,246.322 48.813,246.322 48.9281,246.322 49.0433,246.322 49.1585,246.322 49.2736,246.322 49.3888,246.322 49.5039,246.322 49.6191,246.322 
 49.7343,246.322 49.8494,246.322 49.9646,246.322 50.0797,246.322 50.1949,246.322 50.31,246.322 50.4252,246.322 50.5404,246.322 50.6555,246.322 50.7707,246.322 
 50.8858,246.322 51.001,246.322 51.1161,246.322 51.2313,246.322 51.3465,246.322 51.4616,246.322 51.5768,246.322 51.6919,246.322 51.8071,246.322 51.9222,246.322 
 52.0374,246.322 52.1526,246.322 52.2677,246.322 52.3829,246.322 52.498,246.322 52.6132,246.322 52.7283,246.322 52.8435,246.322 52.9587,246.322 53.0738,246.322 
 53.189,246.322 53.3041,246.322 53.4193,246.322 53.5344,246.322 53.6496,246.322 53.7648,246.322 53.8799,246.322 53.9951,246.322 54.1102,246.322 54.2254,246.322 
 54.3406,246.322 54.4557,246.322 54.5709,246.322 54.686,246.322 54.8012,246.322 54.9163,246.322 55.0315,246.322 55.1467,246.322 55.2618,246.322 55.377,246.322 
 55.4921,246.322 55.6073,246.322 55.7224,246.322 55.8376,246.322 55.9528,246.322 56.0679,246.322 56.1831,246.322 56.2982,246.322 56.4134,246.322 56.5285,246.322 
 56.6437,246.322 56.7589,246.322 56.874,246.322 56.9892,246.322 57.1043,246.322 57.2195,246.322 57.3346,246.322 57.4498,246.322 57.565,246.322 57.6801,246.322 
 57.7953,246.322 57.9104,246.322 58.0256,246.322 58.1407,246.322 58.2559,246.322 58.3711,246.322 58.4862,246.322 58.6014,246.322 58.7165,246.322 58.8317,246.322 
 58.9469,246.322 59.062,246.322 59.1772,246.322 59.2923,246.322 59.4075,246.322 59.5226,246.322 59.6378,246.322 59.753,246.322 59.8681,246.322 59.9833,246.322 
 60.0984,246.322 60.2136,246.322 60.3287,246.322 60.4439,246.322 60.5591,246.322 60.6742,246.322 60.7894,246.322 60.9045,246.322 61.0197,246.322 61.1348,246.322 
 61.25,246.322 61.3652,246.322 61.4803,246.322 61.5955,246.322 61.7106,246.322 61.8258,246.322 61.9409,246.322 62.0561,246.322 62.1713,246.322 62.2864,246.322 
 62.4016,246.322 62.5167,246.322 62.6319,246.322 62.747,246.322 62.8622,246.322 62.9774,246.322 63.0925,246.322 63.2077,246.322 63.3228,246.322 63.438,246.322 
 63.5531,246.322 63.6683,246.322 63.7835,246.322 63.8986,246.322 64.0138,246.322 64.1289,246.322 64.2441,246.322 64.3593,246.322 64.4744,246.322 64.5896,246.322 
 64.7047,246.322 64.8199,246.322 64.935,246.322 65.0502,246.322 65.1654,246.322 65.2805,246.322 6

In [121]:
plot(x_plot_log,y_plot_log)
plot!(sx,sy,st=:scatter)
#調整用
tuney=[0.0 0.5]
tunex=[0.5 0.1]
#plot!(real_sx,real_sy,st=:scatter)
plot!(tunex,tuney, st=:scatter)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
<polyline clip-path="url(#clip1302)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 148.469,303.863 148.469,303.863 148.469,303.863 148.469,303.863 148.469,303.863 148.469,303.863 148.469,303.863 148.469,303.863 148.469,303.863 148.469,303.863 
 148.469,303.863 148.469,303.863 148.469,303.863 148.469,303.863 148.469,303.863 148.47,303.863 148.47,303.863 148.47,303.863 148.47,303.863 148.47,303.863 
 148.47,303.863 148.47,303.863 148.47,303.863 148.47,303.863 148.47,303.863 148.47,303.864 148.47,303.864 148.47,303.864 148.47,303.864 148.47,303.864 
 148.47,303.864 148.47,303.865 148.47,303.865 148.47,303.865 148.47,303.865 148.47,303.866 148.47,303.866 148.47,303.866 148.47,303.867 148.471,303.867 
 148.471,303.868 148.471,303.868 148.471,303.869 148.471,303.869 148.471,303.87 148.471,303.87 148.471,303.871 148.472,303.872 148.472,303.872 148.472,303.873 
 148.472,303.874 148.472,303.874 148.473,303.875 148.473,303.876 148.473,303.877 148.473,303.878 148.474,303.879 148.474,303.88 148.474,303.881 148.475,303.882 
 148.475,303.883 148.475,303.885 148.476,303.886 148.476,303.887 148.476,303.889 148.477,303.89 148.477,303.892 148.478,303.893 148.478,303.895 148.479,303.896 
 148.479,303.898 148.48,303.9 148.48,303.901 148.481,303.903 148.481,303.905 148.482,303.907 148.482,303.909 148.483,303.911 148.484,303.913 148.484,303.915 
 148.485,303.918 148.486,303.92 148.486,303.922 148.487,303.925 148.488,303.927 148.489,303.93 148.49,303.932 148.49,303.935 148.491,303.938 148.492,303.941 
 148.493,303.944 148.494,303.946 148.495,303.949 148.496,303.952 148.497,303.956 148.498,303.959 148.499,303.962 148.5,303.965 148.501,303.969 148.503,303.972 
 148.504,303.976 148.505,303.979 148.506,303.983 148.508,303.987 148.509,303.99 148.51,303.994 148.512,303.998 148.513,304.002 148.514,304.006 148.516,304.01 
 148.517,304.015 148.519,304.019 148.52,304.023 148.522,304.028 148.524,304.032 148.525,304.036 148.527,304.041 148.529,304.046 148.53,304.05 148.532,304.055 
 148.534,304.06 148.536,304.065 148.538,304.07 148.54,304.075 148.542,304.08 148.544,304.085 148.546,304.091 148.548,304.096 148.55,304.101 148.552,304.107 
 148.554,304.112 148.556,304.118 148.558,304.123 148.561,304.129 148.563,304.135 148.565,304.141 148.568,304.147 148.57,304.153 148.573,304.159 148.575,304.165 
 148.578,304.171 148.58,304.177 148.583,304.183 148.585,304.19 148.588,304.196 148.591,304.202 148.594,304.209 148.596,304.216 148.599,304.222 148.602,304.229 
 148.605,304.236 148.608,304.242 148.611,304.249 148.614,304.256 148.617,304.263 148.62,304.27 148.623,304.277 148.627,304.284 148.63,304.291 148.633,304.299 
 148.637,304.306 148.64,304.313 148.643,304.321 148.647,304.328 148.65,304.335 148.654,304.343 148.658,304.35 148.661,304.358 148.665,304.366 148.669,304.373 
 148.672,304.381 148.676,304.389 148.68,304.397 148.684,304.405 148.688,304.413 148.692,304.421 148.696,304.429 148.7,304.437 148.704,304.445 148.708,304.453 
 148.713,304.461 148.717,304.469 148.721,304.478 148.726,304.486 148.73,304.494 148.734,304.503 148.739,304.511 148.744,304.519 148.748,304.528 148.753,304.536 
 148.757,304.545 148.762,304.554 148.767,304.562 148.772,304.571 148.777,304.579 148.782,304.588 148.787,304.597 148.792,304.606 148.797,304.614 148.802,304.623 
 148.807,304.632 148.812,304.641 148.818,304.65 148.823,304.659 148.828,304.668 148.834,304.676 148.839,304.685 148.845,304.694 148.85,304.703 148.856,304.713 
 148.862,304.722 148.867,304.731 148.873,304.74 148.879,304.749 148.885,304.758 148.891,304.767 148.897,304.776 148.903,304.785 148.909,304.795 148.915,304.804 
 148.921,304.813 148.928,304.822 148.934,304.831 148.94,304.841 148.947,304.85 148.953,304.859 148.96,304.868 148.966,304.878 148.973,304.887 148.98,304.8

In [122]:
plot(t,py_ref)
plot!(t,py)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
<polyline clip-path="url(#clip1402)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,135.512 39.4852,135.512 39.6004,135.512 39.7156,135.512 39.8307,135.512 39.9459,135.512 40.061,135.512 40.1762,135.512 40.2913,135.512 40.4065,135.512 
 40.5217,135.512 40.6368,135.512 40.752,135.512 40.8671,135.512 40.9823,135.512 41.0974,135.512 41.2126,135.512 41.3278,135.512 41.4429,135.512 41.5581,135.512 
 41.6732,135.512 41.7884,135.512 41.9035,135.512 42.0187,135.512 42.1339,135.512 42.249,135.512 42.3642,135.512 42.4793,135.512 42.5945,135.512 42.7096,135.512 
 42.8248,135.512 42.94,135.512 43.0551,135.512 43.1703,135.512 43.2854,135.512 43.4006,135.512 43.5157,135.512 43.6309,135.512 43.7461,135.512 43.8612,135.512 
 43.9764,135.512 44.0915,135.512 44.2067,135.512 44.3219,135.512 44.437,135.512 44.5522,135.512 44.6673,135.512 44.7825,135.512 44.8976,135.512 45.0128,135.512 
 45.128,135.512 45.2431,135.512 45.3583,135.512 45.4734,135.512 45.5886,135.512 45.7037,135.512 45.8189,135.512 45.9341,135.512 46.0492,135.512 46.1644,135.512 
 46.2795,135.512 46.3947,135.512 46.5098,135.512 46.625,135.512 46.7402,135.512 46.8553,135.512 46.9705,135.512 47.0856,135.512 47.2008,135.512 47.3159,135.512 
 47.4311,135.512 47.5463,135.512 47.6614,135.512 47.7766,135.512 47.8917,135.512 48.0069,135.512 48.122,135.512 48.2372,135.512 48.3524,135.512 48.4675,135.512 
 48.5827,135.512 48.6978,135.512 48.813,135.512 48.9281,135.512 49.0433,135.512 49.1585,135.512 49.2736,135.512 49.3888,135.512 49.5039,135.512 49.6191,135.512 
 49.7343,135.512 49.8494,135.512 49.9646,135.512 50.0797,135.512 50.1949,135.512 50.31,135.512 50.4252,135.512 50.5404,135.512 50.6555,135.512 50.7707,135.512 
 50.8858,135.512 51.001,135.512 51.1161,135.512 51.2313,135.512 51.3465,135.512 51.4616,135.512 51.5768,135.512 51.6919,135.512 51.8071,135.512 51.9222,135.512 
 52.0374,135.512 52.1526,135.512 52.2677,135.512 52.3829,135.512 52.498,135.512 52.6132,135.512 52.7283,135.512 52.8435,135.512 52.9587,135.512 53.0738,135.512 
 53.189,135.512 53.3041,135.512 53.4193,135.512 53.5344,135.512 53.6496,135.512 53.7648,135.512 53.8799,135.512 53.9951,135.512 54.1102,135.512 54.2254,135.512 
 54.3406,135.512 54.4557,135.512 54.5709,135.512 54.686,135.512 54.8012,135.512 54.9163,135.512 55.0315,135.512 55.1467,135.512 55.2618,135.512 55.377,135.512 
 55.4921,135.512 55.6073,135.512 55.7224,135.512 55.8376,135.512 55.9528,135.512 56.0679,135.512 56.1831,135.512 56.2982,135.512 56.4134,135.512 56.5285,135.512 
 56.6437,135.512 56.7589,135.512 56.874,135.512 56.9892,135.512 57.1043,135.512 57.2195,135.512 57.3346,135.512 57.4498,135.512 57.565,135.512 57.6801,135.512 
 57.7953,135.512 57.9104,135.512 58.0256,135.512 58.1407,135.512 58.2559,135.512 58.3711,135.512 58.4862,135.512 58.6014,135.512 58.7165,135.512 58.8317,135.512 
 58.9469,135.512 59.062,135.512 59.1772,135.512 59.2923,135.512 59.4075,135.512 59.5226,135.512 59.6378,135.512 59.753,135.512 59.8681,135.512 59.9833,135.512 
 60.0984,135.512 60.2136,135.512 60.3287,135.512 60.4439,135.512 60.5591,135.512 60.6742,135.512 60.7894,135.512 60.9045,135.512 61.0197,135.512 61.1348,135.512 
 61.25,135.512 61.3652,135.512 61.4803,135.512 61.5955,135.512 61.7106,135.512 61.8258,135.512 61.9409,135.512 62.0561,135.512 62.1713,135.512 62.2864,135.512 
 62.4016,135.512 62.5167,135.512 62.6319,135.512 62.747,135.512 62.8622,135.512 62.9774,135.512 63.0925,135.512 63.2077,135.512 63.3228,135.512 63.438,135.512 
 63.5531,135.512 63.6683,135.512 63.7835,135.512 63.8986,135.512 64.0138,135.512 64.1289,135.512 64.2441,135.512 64.3593,135.512 64.4744,135.512 64.5896,135.512 
 64.7047,135.512 64.8199,135.512 64.935,135.512 65.0502,135.512 65.1654,135.512 65.2805,135.512 65.39